In [ ]:
# 本模块：文章聚类与评价

In [149]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score
nmi = lambda x, y: normalized_mutual_info_score(x, y, average_method='arithmetic')

In [150]:
data = pd.read_csv('data.csv')
ori = data['topic'].values
orig = np.zeros(2225)
dic = {'business': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'tech': 4}
for id in range(len(ori)):
    orig[id] = dic[ori[id]]
orig = orig.astype('int')

In [151]:
# 计算纯度purity和互信息nmi
def clustering(table):
    # 聚类
    cluster = KMeans(n_clusters=5)
    cluster.fit(table)
    pred = cluster.predict(table)
    nir = np.zeros((5, 5))
    ni = np.zeros(5)
    nr = np.zeros(5)
    for i in range(table.shape[0]):
        nir[orig[i]][pred[i]] += 1
        ni[orig[i]] += 1
        nr[pred[i]] += 1
    f = np.zeros((5, 5))
    for i in range(5):
        for r in range(5):
            precision = nir[i][r]/nr[r]
            recall = nir[i][r]/ni[i]
            if abs(precision+recall) < 1e-6:
                f[i][r] = 0
            else:
                f[i][r] = 2*precision*recall/(precision+recall)
    purity = 0
    for r in range(5):
        maxnir = 0
        for i in range(5):
            maxnir = max(maxnir, nir[i][r])
        purity += (maxnir/table.shape[0])
    Fv = 0
    for i in range(5):
        maxf = 0
        for r in range(5):
            maxf = max(maxf, f[i][r])
        Fv += (maxf/table.shape[0])
    return purity, Fv, nmi(orig, pred)

In [159]:
# original 原始数据
table = pd.read_csv("newsvec.csv")
table = table.drop(columns=['Unnamed: 0'])
p1, n1 = 0, 0
for i in range(5):
    purity, Fv, Nmi = clustering(table)
    p1 += purity
    n1 += Nmi
print("Purity: ", p1/5, "NMI: ", n1/5)

Purity:  0.575370786516854 NMI:  0.4907050722497015


In [158]:
# pca 降至50维
table = pd.read_csv("vecpca2.csv")
table = table.drop(columns=['Unnamed: 0'])
p2, n2 = 0, 0
for i in range(5):
    purity, Fv, Nmi = clustering(table)
    p2 += purity
    n2 += Nmi
print("Purity: ", p2/5, "NMI: ", n2/5)

Purity:  0.624629213483146 NMI:  0.5715147520673607


In [157]:
# pca 降至10维
table = pd.read_csv("vecpca.csv")
table = table.drop(columns=['Unnamed: 0'])
p3, n3 = 0, 0
for i in range(5):
    purity, Fv, Nmi = clustering(table)
    p3 += purity
    n3 += Nmi
print("Purity: ", p3/5, "NMI: ", n3/5)

Purity:  0.6893483146067416 NMI:  0.6352787216021816
